# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [9]:
# Write your code below.
from dotenv import load_dotenv
import os

load_dotenv()
price_data_path = os.getenv("PRICE_DATA")
print("PRICE_DATA path:", price_data_path)




PRICE_DATA path: ../../05_src/data/prices/


In [10]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [11]:
from glob import glob
import os

parquet_files = glob(os.path.join(price_data_path, "**", "*.parquet"), recursive=True)
print("Found Parquet files:", parquet_files)


Found Parquet files: ['../../05_src/data/prices\\ABEV\\ABEV_1997\\part.0.parquet', '../../05_src/data/prices\\ABEV\\ABEV_1997\\part.1.parquet', '../../05_src/data/prices\\ABEV\\ABEV_1998\\part.0.parquet', '../../05_src/data/prices\\ABEV\\ABEV_1998\\part.1.parquet', '../../05_src/data/prices\\ABEV\\ABEV_1999\\part.0.parquet', '../../05_src/data/prices\\ABEV\\ABEV_1999\\part.1.parquet', '../../05_src/data/prices\\ABEV\\ABEV_2000\\part.0.parquet', '../../05_src/data/prices\\ABEV\\ABEV_2000\\part.1.parquet', '../../05_src/data/prices\\ABEV\\ABEV_2001\\part.0.parquet', '../../05_src/data/prices\\ABEV\\ABEV_2001\\part.1.parquet', '../../05_src/data/prices\\ABEV\\ABEV_2002\\part.0.parquet', '../../05_src/data/prices\\ABEV\\ABEV_2002\\part.1.parquet', '../../05_src/data/prices\\ABEV\\ABEV_2003\\part.0.parquet', '../../05_src/data/prices\\ABEV\\ABEV_2003\\part.1.parquet', '../../05_src/data/prices\\ABEV\\ABEV_2004\\part.0.parquet', '../../05_src/data/prices\\ABEV\\ABEV_2004\\part.1.parquet', '.

In [12]:
import os
from glob import glob

# Write your code below.
parquet_files = glob(os.path.join(price_data_path, "**", "*.parquet"), recursive=True)
print("Found Parquet files:", parquet_files)




Found Parquet files: ['../../05_src/data/prices\\ABEV\\ABEV_1997\\part.0.parquet', '../../05_src/data/prices\\ABEV\\ABEV_1997\\part.1.parquet', '../../05_src/data/prices\\ABEV\\ABEV_1998\\part.0.parquet', '../../05_src/data/prices\\ABEV\\ABEV_1998\\part.1.parquet', '../../05_src/data/prices\\ABEV\\ABEV_1999\\part.0.parquet', '../../05_src/data/prices\\ABEV\\ABEV_1999\\part.1.parquet', '../../05_src/data/prices\\ABEV\\ABEV_2000\\part.0.parquet', '../../05_src/data/prices\\ABEV\\ABEV_2000\\part.1.parquet', '../../05_src/data/prices\\ABEV\\ABEV_2001\\part.0.parquet', '../../05_src/data/prices\\ABEV\\ABEV_2001\\part.1.parquet', '../../05_src/data/prices\\ABEV\\ABEV_2002\\part.0.parquet', '../../05_src/data/prices\\ABEV\\ABEV_2002\\part.1.parquet', '../../05_src/data/prices\\ABEV\\ABEV_2003\\part.0.parquet', '../../05_src/data/prices\\ABEV\\ABEV_2003\\part.1.parquet', '../../05_src/data/prices\\ABEV\\ABEV_2004\\part.0.parquet', '../../05_src/data/prices\\ABEV\\ABEV_2004\\part.1.parquet', '.

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [ ]:
# Write your code below.
import dask.dataframe as dd

# Loading the Parquet files
ddf = dd.read_parquet(parquet_files)

# Adding lag features
ddf["Close_lag_1"] = ddf["Close"].shift(1)
ddf["Adj_Close_lag_1"] = ddf["Adj Close"].shift(1)

# Adding returns
ddf["returns"] = (ddf["Close"] / ddf["Close_lag_1"]) - 1

# Adding high-low range
ddf["hi_lo_range"] = ddf["High"] - ddf["Low"]

# Assigning to dd_feat
dd_feat = ddf





In [14]:
dd_feat.head()


,Date,Open,High,Low,Close,Adj Close,Volume,source,ticker,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range
0,1997-03-05,0.4500,0.4500,0.4500,0.4500,0.007347,15000,ABEV.csv,ABEV,1997,NaN,NaN,NaN,0.0
1,1997-03-06,0.4500,0.4500,0.4500,0.4500,0.007347,0,ABEV.csv,ABEV,1997,0.4500,0.007347,0.000000,0.0
2,1997-03-07,0.4500,0.4500,0.4500,0.4500,0.007347,0,ABEV.csv,ABEV,1997,0.4500,0.007347,0.000000,0.0
3,1997-03-10,0.4875,0.4875,0.4875,0.4875,0.007959,22500,ABEV.csv,ABEV,1997,0.4500,0.007347,0.083333,0.0
4,1997-03-11,0.4875,0.4875,0.4875,0.4875,0.007959,0,ABEV.csv,ABEV,1997,0.4875,0.007959,0.000000,0.0


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [ ]:
# Write your code below.

# Converting to pandas
pdf = dd_feat.compute()

# Adding 10-day moving average of returns
pdf["returns_ma_10"] = pdf["returns"].rolling(10).mean()

# Previewing the result
pdf[["Date", "returns", "returns_ma_10"]].head(15)




,Date,returns,returns_ma_10
0,1997-03-05,NaN,NaN
1,1997-03-06,0.000000,NaN
2,1997-03-07,0.000000,NaN
3,1997-03-10,0.083333,NaN
4,1997-03-11,0.000000,NaN
5,1997-03-12,0.000000,NaN
6,1997-03-13,0.000000,NaN
7,1997-03-14,0.000000,NaN
8,1997-03-17,0.000000,NaN
9,1997-03-18,0.000000,NaN


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

In [16]:
# Question : Was it necessary to convert to pandas to calculate the moving average return?

#ANSWER: 
# Yes, it was necessary to convert to pandas to calculate the moving average return because
# Dask does not currently support the .rolling() method for window-based operations.
# Rolling calculations require sequential access to rows, which pandas handles efficiently in-memory.


# Would it have been better to do it in Dask? Why?

# It would not have been better to do it in Dask for this specific task. While Dask is great for
# parallel and distributed processing of large datasets, it lacks native support for rolling window
# operations. Simulating this in Dask would be more complex and less efficient. Therefore, converting
# to pandas was the most practical and performant choice for calculating the moving average.


## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.